# TP : Jeu du domineering

Le domineering est un jeu de plateau où le joueur 0 place un domino vertical et le joueur 1 un domino horizontal. Un joueur qui ne peut plus jouer perd.

Voici un exemple de partie (de gauche à droite) où le joueur 1 gagne :
<center><img src=https://raw.githubusercontent.com/fortierq/tikz-pdf/main/jeux/domineering/ex.png width=600></center>

Une configuration est représentée par une matrice (-1 = vide, 0 = joueur 0, 1 = joueur 1)


 Écrire une fonction `grille_vide(n, p)` qui renvoie une grille de taille $n \times p$ vide (remplie de $-1$).


In [1]:
def grille_vide(n:int, p:int) -> list : 
    return [[-1] * p for _ in range(n)]

In [2]:
grille_vide(3, 4)

[[-1, -1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1]]


 Écrire une fonction `coups_possibles(v, joueur)` qui prend en entrée une configuration `v` et qui renvoie la liste des positions $(i, j)$ où le joueur `joueur` peut placer son domino (le joueur $0$ place un domino en position $(i, j)$ et $(i+1, j)$ et le joueur $1$ en position $(i, j)$ et $(i, j+1)$).


In [10]:
def coups_possibles(v, joueur):
    position = []
    if joueur == 0:
        for i in range(len(v) - 1):
            for j in range(len(v[0])):
                if v[i][j] == -1 and v[i + 1][j] == -1:
                    position.append((i, j))
    elif joueur == 1:
        for i in range(len(v)):
            for j in range(len(v[0]) - 1):
                if v[i][j] == -1 and v[i][j + 1] == -1:
                    position.append((i, j))
    return position

In [11]:
print(coups_possibles(grille_vide(3, 4), 0))
print(coups_possibles(grille_vide(3, 4), 1))

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]



 Écrire une fonction `strategie_aleatoire(v, joueur)` qui prend en entrée une configuration `v` et qui renvoie un coup choisi au hasard parmi les coups possibles.  
On pourra utiliser la fonction `random.choice(L)` qui renvoie un élément au hasard dans `L` (en n'oubliant pas `import random`).


In [12]:
import random as rd 

def strategie_aleatoire(v, joueur):
    return rd.choice(coups_possibles(v, joueur))

In [13]:
strategie_aleatoire(grille_vide(3, 4), 0) # coup aléatoire

(0, 0)


 Écrire une fonction `placer(v, i, j, joueur)` qui prend en entrée une configuration `v`, une position $(i, j)$ et un joueur `joueur` et qui modifie `v` en plaçant le domino du joueur `joueur` à la position $(i, j)$.  
Plus précisément, si `joueur == 0`, alors on place un domino sur les cases $(i, j)$ et $(i + 1, j)$, et si `joueur == 1`, alors on place un domino sur les cases $(i, j)$ et $(i, j + 1)$.


In [14]:
def placer(v, i, j, joueur):
    place = [(i + 1, j), (i, j + 1)]
    v[i][j] = joueur
    v[place[joueur][0]][place[joueur][1]] = joueur



 Écrire une fonction `retirer(v, i, j, joueur)` qui prend en entrée une configuration `v`, une position $(i, j)$ et un joueur `joueur` et qui modifie `v` en retirant le domino du joueur `joueur` à la position $(i, j)$.


In [15]:
def retirer(v, i, j, joueur):
    place = [(i + 1, j), (i, j + 1)]
    v[i][j] = -1
    v[place[joueur][0]][place[joueur][1]] = -1


 Écrire une fonction `jeu(strategie0, strategie1, n, p)` qui prend en entrée deux stratégies et qui renvoie le joueur qui gagne.  
`strategie0(v, 0)` renvoie un couple $(i, j)$ correspondant au coup du joueur 0, et `strategie1(v, 1)` renvoie le coup du joueur 1.  
Il faut donc partir d'une grille de taille $n\times p$ et faire jouer les joueurs chacun son tour avec leur stratégie, jusqu'à ce qu'un joueur ne puisse plus jouer (qui est donc le perdant).


In [21]:
def jeu(strategie0, strategie1, n, p):
    strategie = (strategie0, strategie1)
    v = grille_vide(n, p)
    joueur = 0
    while True:
        coups = coups_possibles(v, joueur)
        if len(coups) == 0:
            return joueur ^ 1
        coup = strategie[joueur](v, joueur)
        placer(v, coup[0], coup[1], joueur)
        joueur ^= 1

In [22]:
jeu(strategie_aleatoire, strategie_aleatoire, 3, 4) # 0 ou 1

0


 Écrire une fonction `statistiques(strategie0, strategie1, n, p, nb_parties)` qui prend en entrée deux stratégies, qui appelle `nb_parties` fois la fonction `jeu` et qui renvoie le nombre de parties gagnées par chaque joueur.  
Tester avec différentes tailles initiales de la grille.


In [23]:
def statistiques(strategie0, strategie1, n, p, nb_parties):
    stat = [0, 0]
    for _ in range(nb_parties):
        gagnant = jeu(strategie0, strategie1, n, p)
        stat[gagnant] += 1
    return stat

In [24]:
statistiques(strategie_aleatoire, strategie_aleatoire, 3, 10, 1000)

[391, 609]


 Écrire une fonction `h1(v)` renvoyant la différence entre le nombre de cases libres pour le joueur 0 et le nombre de cases libres pour le joueur 1. On renverra $\infty$ (`float('inf')`) si le joueur 0 a gagné et $-\infty$ (`float('-inf')`) si le joueur 1 a gagné.


In [84]:
def h1(v):
    pos_0 = len(coups_possibles(v, 0))
    pos_1 = len(coups_possibles(v, 1))
    if pos_0 == 0:
        return float("-inf")
    elif pos_1 == 0:
        return float("inf")
    else:
        return pos_0 - pos_1

def h2(v, joueur):
    pos_0 = len(coups_possibles(v, 0))
    pos_1 = len(coups_possibles(v, 1))
    if pos_0 == 0:
        if joueur == 0:
            return float("-inf")
        else:
            return float("inf")
    elif pos_1 == 0:
        if joueur == 1:
            return float("inf")
        else:
            return float("-inf")
    else:
        return pos_0 - pos_1

In [85]:
h1(grille_vide(3, 2))

1

L'**algorithme min-max** considère, depuis la position en cours, toutes les positions atteignables après $p$ coups et conserver celle ayant la meilleure heuristique. 

Puis on donne récursivement une valeur à chaque sommet `v` de l'arbre :  
- si `v` est à profondeur $0$, on renvoie l'heuristique de `v`
- sinon, si le joueur $0$ doit jouer, on renvoie la valeur maximum des successeurs de `v` (le joueur $0$ veut maximiser sa valeur)
- sinon, si le joueur $1$ doit jouer, on renvoie la valeur minimum des successeurs de `v` (le joueur $1$ veut minimiser la valeur de son adversaire)

On choisit le coup qui donne la meilleure valeur parmi les coups possibles.

<center><img src=https://raw.githubusercontent.com/fortierq/tikz-pdf/main/jeux/domineering/arbre/arbre3.png width=100%></center>


 Écrire une fonction `minmax(v, joueur, profondeur, h)` qui prend en entrée une configuration `v`, un joueur `joueur`, une profondeur `profondeur` et une fonction heuristique `h` et qui renvoie un couple `(valeur, coup)` où `coup` est le meilleur coup à jouer et `valeur` est sa valeur.


In [86]:
def minmax(v, joueur, profondeur, h): # renvoie (valeur, coup)
    coups = coups_possibles(v, joueur)
    if profondeur == 0 or len(coups) == 0:
        return h(v, joueur), None
    else:
        if joueur == 0:
            meilleur = float("-inf"), coups[0]
            for coup in coups:
                # jouer un coup (avec la fonction placer)
                placer(v, coup[0], coup[1], joueur)
                res = minmax(v, joueur ^ 1, profondeur - 1, h)[0], coup
                retirer(v, coup[0], coup[1], joueur)
                meilleur = max(meilleur, res)
            return meilleur
        elif joueur == 1:
            meilleur = float("inf"), coups[0]
            for coup in coups:
                placer(v, coup[0], coup[1], joueur)
                res = minmax(v, joueur ^ 1, profondeur - 1, h)[0], coup
                retirer(v, coup[0], coup[1], joueur)
                meilleur = min(meilleur, res)
            return meilleur


 En déduire une fonction `strategie_minmax(v, joueur)` qui prend en entrée une configuration `v` et un joueur `joueur` et qui renvoie le meilleur coup à jouer selon l'algorithme min-max avec l'heuristique `h1` et, par exemple, une profondeur de 2.  
Tester avec la fonction ci-dessous pour vérifier que la stratégie min-max est meilleure que la stratégie aléatoire.


In [92]:
def strategie_minmax(v, joueur):
    return minmax(v, joueur, 3, h2)[1]

def strategie_minmax1(v, joueur):
    return minmax(v, joueur, 3, h2)[1]

def strategie_minmax2(v, joueur):
    return minmax(v, joueur, 1, h2)[1]

grille_exemple = [
    [-1, 0, -1, -1],
    [-1, 0, 1, 1],
    [1, 1, -1, -1],
    [-1, -1, -1, -1]
]

strategie_minmax(grille_exemple, 0)

(2, 3)

In [96]:
statistiques(strategie_minmax, strategie_minmax, 5, 5, 1)

[0, 1]

In [45]:
stat = [0, 0]
n = 10
for _ in range(n):
    sim = statistiques(strategie_minmax, strategie_aleatoire, 5, 5, 100)
    stat[1] += sim[1]
    stat[0] += sim[0]

a, b = stat[0] / n, stat[1] / n


In [46]:
print(a, b)

94.5 5.5
